In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras

### __Deep Learning workflow:__
<font size=3>
    
1. Import and data pre-processing;   
2. Neural network modeling;
3. Model compilation;
4. Train and validation;
5. Final training;
6. Test evaluation;
7. Saving the model.

### __1. Import and data pre-processing:__
<font size=3>
    
1.1 Import data;\
1.2 Data visualization;\
1.3 Feature engineering;\
1.4 Data shuffling;\
1.5 Train, validation, and test tensor divition.

Our first task will classify _normal_ and _abnormal_ orthopedic diagnoses from biomechanical features. Our data can be downloaded from the [Kaggle datasets](https://www.kaggle.com/datasets/uciml/biomechanical-features-of-orthopedic-patients?resource=download).

In [ ]:
df = pd.read_csv("dataset/column_2C_weka.csv")

df.head()

In [ ]:
# how many classes do we have?

n_normal = len(df[df['class'] == 'Normal'])
n_abnormal = len(df[df['class'] == 'Abnormal'])

n_normal, n_abnormal

In [ ]:
# data shuffling:
i = np.random.permutation(len(df))

df = df.iloc[i, :]

df.head()

In [ ]:
# features data:
df.iloc[:, 0:6].head()

In [ ]:
x = df.iloc[:, 0:6].to_numpy()
x

In [ ]:
# target data:
df.iloc[:, -1:].head()

In [ ]:
def label_fn(x):
    if x == 'Abnormal': return 1.0
    else: return 0.0

y = df.iloc[:, -1].apply(label_fn).to_numpy()
y

In [ ]:
x.shape, y.shape

In [ ]:
print("y:", y.shape)

# since x array has 2 dimensions, we'll expand y dimensions:
y = np.expand_dims(y, axis=1)

print("y:", y.shape)

In [ ]:
# normalizing the data:
print(f"x before norm: max = {x.max():.2f}, min = {x.min():.2f}")

x /= abs(x).max() # normalization: x = x/max(abs(x))

print(f"x after norm: max = {x.max():.2f}, min = {x.min():.2f}")

In [ ]:
# splitting the dataset into train, validation and test:
N_samples, N_features = x.shape

N_train = int(0.70*N_samples) # 80%

N_val = int(0.2*N_samples) # 20%

N_test = N_samples - (N_train + N_val) # 10%

print(f"N-samples = {N_samples}, N-train = {N_train}, N-val = {N_val}, N-test = {N_test}")

print(N_samples == N_train + N_val + N_test)

In [ ]:
x_train = x[:N_train]
y_train = y[:N_train]

x_val = x[N_train:N_train+N_val]
y_val = y[N_train:N_train+N_val]

x_test = x[N_train+N_val:]
y_test = y[N_train+N_val:]

print(f"x-train:{x_train.shape}, y-train:{y_train.shape}")
print(f"x-val:{x_val.shape}, y-val:{y_val.shape}")
print(f"x-test:{x_test.shape}, y-test:{y_test.shape}")

In [ ]:
# we can delete unnecessary tansors and dataframe:

del df, x, y

### __2. Neural network modeling:__
<font size=3>
    
2.1 Define initial layer's shape;\
2.2 Define output layer's shape and its [activation function](https://keras.io/api/layers/activations/);\
2.3 Define hidden layers.

[Checkout Keras API](https://keras.io/guides/functional_api/).

In [ ]:
In = keras.Input(shape=(x_train.shape[1],))

x = keras.layers.Dense(50, activation='sigmoid')(In)

Out = keras.layers.Dense(1, activation='sigmoid')(x)

model = keras.Model(inputs=In, outputs=Out)

model.summary()

### __3. Model compilation:__
<font size=3>

3.1 Define [optimizer](https://keras.io/api/optimizers/);\
3.2 Define [loss function](https://keras.io/api/losses/);\
3.3 Define [validation metric](https://keras.io/api/metrics/).


In [ ]:
model.compile(optimizer='SGD', loss="mse", metrics=['acc'])

### __4. Train and validation__
<font size=3>
    
Here, using the training data, the optimizer updates the values of the model's inner parameters (_i.e._, weights, biases, etc.) over the epochs while minimizing/maximizing the loss function. Meanwhile, the model's performance is measured for each epoch using the validation data. At this workflow stage, we model the neural network architecture to avoid [overfitting and underfitting](https://www.geeksforgeeks.org/underfitting-and-overfitting-in-machine-learning/).

__Underfitting__ means a poor NN fitting, _i.e._, the model does not learn well. On the other hand, __overfitting__ occurs when the model fits the training data very well but makes poor predictions with validation data.

__To avoid underfitting__, we need to make the NN more robust - with more layers and neurons - to increase the NN's depth.

__To avoid overfitting__, we have two basic options: __i)__ decrease the number of neurons (or/and layers) - as an analogy, we are decreasing the degree of a polynomial function (check the [figure](https://www.geeksforgeeks.org/underfitting-and-overfitting-in-machine-learning/) again); __ii)__ we can apply a [dropout layer](https://keras.io/api/layers/regularization_layers/dropout/) after the layer with the largest number of neurons.

What is the dropout layer? Dropout _"closes"_ the activation of neurons from the previous layer at random by setting them to zero! When training becomes rigid, we create a type of _"neuroplasticity"_ in the network to form more flexible connections. Check the [paper](https://paperswithcode.com/method/dropout)'s motivation!



In [ ]:
report = model.fit(x=x_train, y=y_train, validation_data=[x_val, y_val], epochs=10)

In [ ]:
report.history

In [ ]:
loss = report.history['loss']
val_loss = report.history['val_loss']

acc = report.history['acc']
val_acc = report.history['val_acc']

epochs = np.linspace(1, len(loss), len(loss))

fig, ax = plt.subplots(1, 2, figsize=(12,4))

ax[0].plot(epochs, loss, label='train')
ax[0].plot(epochs, val_loss, label='val')
ax[0].set_ylabel("MSE")

ax[1].plot(epochs, acc, label='train')
ax[1].plot(epochs, val_acc, label='val')
ax[1].set_ylabel("Accuracy")

for i in range(2):
    ax[i].legend()
    ax[i].set_xlabel('epochs')
    ax[i].grid()
    
plt.show()

### __5. Final training__
<font size=3>

Once the modeling is completed, we concatenate train and validation data to fit again the model.

__Note:__ use the same number of epochs from the previous step.
    

### __6. Test evaluation__:

    6.1 Make the evaluation using the test data;
    6.1 Make some predictions to visualize the results;
   

### __7. Saving the model__:
<font size=3>
    
[Checkout](https://keras.io/api/models/model_saving_apis/).